# Задача 2 任务 2
1.Сгенерируйте, содержащий 50000 32-битных случайных целых чисел, каждое число на отдельной строке.
生成包含 50000 个 32 位随机整数，每个数字在单独的行上。

2.Посчитать, какое суммарное количество простых множителей присутствует при факторизации всех чисел. Например, пусть всего два числа: 6 и 8. 6 = 2 * 3, 8 = 2 * 2 * 2. Ответ 5.
计算因式分解所有数字时存在的质因数总数。例如，假设只有两个数字：6 和 8。6 = 2 * 3,8 = 2 * 2 * 2。答5.

3.Реализовать подсчет 实现计数

  простым последовательным алгоритмом
  简单的顺序算法
  многопоточно (на CPython - multiprocessing), с использованием примитивов синхронизации
  多线程（CPython 上的多处理），使用同步原语
  с помощью Ray/Dask/PySpark
  使用 Ray/Dask/PySpark
  (*) с использованием Go (go-рутин) / корутин на Kotlin
  （*） 在 Kotlin 中使用 Go （go-routine） / 协程
4.Измерить время выполнения для каждого случая.
测量每个案例的执行时间。

# 1.Сгенерируйте, содержащий 50000 32-битных случайных целых чисел, каждое число на отдельной строке.
生成包含 50000 个 32 位随机整数，每个数字在单独的行上。

In [ ]:
import random

with open('data2.txt', 'w') as file:
    for _ in range(50000):
        num = random.randint(-2147483648, 2147483647)
        file.write(f"{num}\n")


# Посчитать, какое суммарное количество простых множителей присутствует при факторизации всех чисел. Например, пусть всего два числа: 6 и 8. 6 = 2 * 3, 8 = 2 * 2 * 2. Ответ 5.простым последовательным алгоритмом
简单的顺序算法
计算因式分解所有数字时存在的质因数总数。例如，假设只有两个数字：6 和 8。6 = 2 * 3,8 = 2 * 2 * 2。答5.

In [7]:
def calculate(num):
    factors = []
    i = 2
    while i * i <= num:
        if num % i == 0:
            factors.append(i)
            num = num // i
        else:
            i += 1
    if num > 1:
        factors.append(num)
    return len(factors)

# 打开包含随机整数的文件
with open('data2.txt', 'r') as file:
    result = 0
    for line in file:
        number = int(line.strip())
        result += calculate(number)

print("总质因数个数 Количество факторов общего качества：", result)


总质因数个数 Количество факторов общего качества： 102898


# многопоточно (на CPython - multiprocessing), с использованием примитивов синхронизации
多线程（CPython 上的多处理），使用同步原语

In [13]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  


def calculate(num):
    factors = []
    i = 2
    while i * i <= num:
        if num % i == 0:
            factors.append(i)
            num = num // i
        else:
            i += 1
    if num > 1:
        factors.append(num)
    return len(factors)  # 返回质因数数量


with open('data2.txt', 'r') as file:
    numbers = [int(line.strip()) for line in file.readlines()]

# 使用ThreadPoolExecutor来处理任务
results = []
with ThreadPoolExecutor(max_workers=2) as executor:
  
    num = {executor.submit(calculate, num): num for num in numbers}  
    # 使用tqdm
    for process in tqdm(as_completed(num), total=len(numbers), desc="Processing numbers"):
        results.append(process.result())



print( sum(results))




Processing numbers: 100%|██████████| 50000/50000 [00:07<00:00, 6574.80it/s] 

102898


# с помощью Dask
使用 Ray/Dask/PySpark

In [14]:
import dask.dataframe as dd

def calculate(num):
    factors = []
    i = 2
    while i * i <= num:
        if num % i == 0:
            factors.append(i)
            num = num // i
        else:
            i += 1
    if num > 1:
        factors.append(num)
    return len(factors)  # 返回质因数数量而不是质因数列表

# 使用 Dask 读取文件
df = dd.read_csv('data2.txt', header=None, names=['number'], dtype={'number': 'int64'}, blocksize=None)

# 打印头部数据以检查是否正确读取
print(df.head())


# 应用函数到每一行，并计算总和
count = df['number'].map(calculate).sum().compute()

print(count)





       number
0  1334259689
1  1108738542
2  -331276288
3 -1742890076
4   571301911
102898


# Измерить время выполнения для каждого случая.
测量每个案例的执行时间。

In [17]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import dask.dataframe as dd

# 简单的顺序算法
def method1():
    def calculate(num):
        factors = []
        i = 2
        while i * i <= num:
            if num % i == 0:
                factors.append(i)
                num = num // i
            else:
                i += 1
        if num > 1:
            factors.append(num)
        return len(factors)

    start_time = time.time()
    result = 0
    with open('data2.txt', 'r') as file:
        for line in file:
            number = int(line.strip())
            result += calculate(number)
    end_time = time.time()
    print("顺序读取时间 Простое последовательное чтение:{:.2f} s".format(end_time - start_time))


# 多线程（CPython 上的多处理），使用同步原语
def method2():
    def calculate(num):
        factors = []
        i = 2
        while i * i <= num:
            if num % i == 0:
                factors.append(i)
                num = num // i
            else:
                i += 1
        if num > 1:
            factors.append(num)
        return len(factors)

    start_time = time.time()
    with open('data2.txt', 'r') as file:
        numbers = [int(line.strip()) for line in file.readlines()]

    results = []
    with ThreadPoolExecutor(max_workers=4) as executor:  # Adjusted to 4 workers
        num = {executor.submit(calculate, num): num for num in numbers}
        for process in as_completed(num):
            results.append(process.result())

    end_time = time.time()
    print("多线程 многопоточно (на CPython - multiprocessing), с использованием примитивов синхронизации:{:.2f} s".format(end_time - start_time))


# 使用 Dask 的方法
def method3():
    def calculate(num):
        factors = []
        i = 2
        while i * i <= num:
            if num % i == 0:
                factors.append(i)
                num = num // i
            else:
                i += 1
        if num > 1:
            factors.append(num)
        return len(factors)

    start_time = time.time()
    df = dd.read_csv('data2.txt', header=None, names=['number'], dtype={'number': 'int64'}, blocksize=None)
    total_factors_count = df['number'].map(calculate).sum().compute()
    end_time = time.time()
    
    print("使用Dask с помощью Ray/Dask/PySpark:{:.2f} s".format(end_time - start_time))


method1()
print('\n')
method2()
print('\n')
method3()


顺序读取时间 Простое последовательное чтение:7.26 s


多线程 многопоточно (на CPython - multiprocessing), с использованием примитивов синхронизации:9.38 s


使用Dask с помощью Ray/Dask/PySpark:7.31 s
